In [2]:
import pandas as pd
import requests
#us is a module that allows conversion of US state names to abrv. and back
import us

In [3]:
r = requests.get("https://gasprices.aaa.com/state-gas-price-averages/", headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"})
results=pd.read_html(r.text)
gas_table=results[0]
gas_table

,State,Regular,Mid-Grade,Premium,Diesel
0,Alaska,$2.457,$2.557,$2.719,$2.413
1,Alabama,$1.820,$2.145,$2.474,$2.274
2,Arkansas,$1.815,$2.103,$2.391,$2.135
3,Arizona,$2.296,$2.569,$2.821,$2.500
4,California,$3.031,$3.208,$3.324,$3.305
5,Colorado,$2.361,$2.674,$2.947,$2.284
6,Connecticut,$2.093,$2.461,$2.676,$2.732
7,District of Columbia,$2.205,$2.680,$2.810,$2.835
8,Delaware,$2.070,$2.428,$2.661,$2.365
9,Florida,$2.021,$2.374,$2.667,$2.429


In [4]:
type(gas_table)

pandas.core.frame.DataFrame

In [5]:
#Make the column headers lower case to match SQL database
gas_table.columns = map(str.lower, gas_table.columns)

In [6]:
#The cost of gas is a string, need to convert
type(gas_table['regular'][1])

str

In [7]:
cols = ['regular','mid-grade','premium','diesel']

In [8]:
#drop state to lower_case
gas_table["state"] = gas_table["state"].str.lower()

In [9]:
for name in cols:
    gas_table[name] = gas_table[name].str.replace("$","")

In [10]:
gas_table.head()

,state,regular,mid-grade,premium,diesel
0,alaska,2.457,2.557,2.719,2.413
1,alabama,1.820,2.145,2.474,2.274
2,arkansas,1.815,2.103,2.391,2.135
3,arizona,2.296,2.569,2.821,2.500
4,california,3.031,3.208,3.324,3.305


In [11]:
#Change to float and round
for name in cols:
    gas_table[name] = gas_table[name].astype(float).round(2)

In [12]:
#convert states to abbr. to match craigslist

state_abbr = []

for i in range(len(gas_table['state'])):
    state = us.states.lookup(gas_table['state'][i])
    state_abbr.append(state.abbr)

In [13]:
#replace the column state with abbr.
gas_table['state'] = state_abbr

In [14]:
#drop state to lower_case
gas_table["state"] = gas_table["state"].str.lower()

In [15]:
gas_table.head()

,state,regular,mid-grade,premium,diesel
0,ak,2.46,2.56,2.72,2.41
1,al,1.82,2.14,2.47,2.27
2,ar,1.82,2.10,2.39,2.13
3,az,2.30,2.57,2.82,2.50
4,ca,3.03,3.21,3.32,3.30


In [16]:
#write the df to csv for importation into sql
gas_table.to_csv('../Resources/final_gas.csv', index=False)